In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import lsst.daf.butler as dB
import lsst.cp.verify.notebooks.utils as utils
import lsst.afw.display as afwDisplay

In [ ]:
# This cell contains parameters that can be automatically set via the papermill package.
# Examples:
#   papermill --prepare-only -p calibType newBias -p cameraName LSSTCam <input> <output>
#   papermill -p interactive False <input> <output>
interactive = True

# Which repository to use.
repository = '/repo/main/'

# Which calibration type to analyse.
calibType = 'bias'

# Which camera the calibration is for.
cameraName = 'LATISS'

# Which display to use.
displayBackend = 'astrowidgets'

# Which collection the calibration was constructed in.
genCollection = 'u/czw/DM-28920/biasGen.20210702a'

# Which collection containing the verification outputs.
verifyCollection = 'u/czw/DM-28920/verifyBias.20210702a'

In [ ]:
# Get butler and camera
butler = dB.Butler(repository, collections=[verifyCollection, genCollection])
camera = butler.get('camera', instrument=cameraName)

In [ ]:
# Get Run Statistics
runStats = butler.get('verifyBiasStats', instrument=cameraName)
runSuccess = runStats.pop('SUCCESS')

In [ ]:
# Display summary table of tests and failure counts.
utils.failureTable(runStats)

In [ ]:
# Plot focal plane, and display number of amp-level failures per detector.
utils.plotFailures(runStats, camera, scaleFactor=8)

In [ ]:
# This cell may be easier to follow in a new view via the
#     "Create New View for Output" right-click menu.  
afwDisplay.setDefaultBackend(displayBackend)
display = afwDisplay.Display(dims=(1000, 1000))
display.embed()

In [ ]:
# View calibration images.
viewCalibs = interactive
if viewCalibs:
    continueDisplay = True
    for detector in camera:
        detectorId = detector.getId()
        calib = butler.get(calibType, instrument=cameraName, detector=detectorId)
        calibArray = calib.getImage().getArray()

        # Get simple stats
        q25, q50, q75 = np.percentile(calibArray.flatten(), [25, 50, 75])
        sigma = 0.74 * (q75 - q25)
        print(f"Detector: {detector.getName()} Median: {q50}   Stdev: {sigma}")

        display.mtv(calib)
        display._scale('linear', (q50 - 3.0 * sigma), (q50 + 3.0* sigma), "")

        continueDisplay, skipNumber = utils.interactiveBlock(f"{calibType} {detector.getName()}", 
                                                             {})
        if continueDisplay is False:
            break

In [ ]:
# This block allows the residual images to be examined.
blinkResiduals = interactive
if blinkResiduals:
    continueDisplay = True
    skipNumber = 0
    for exposureId, stats in runStats.items():
        for detector in camera:
            if skipNumber > 0:
                skipNumber -= 1
                continue

            detId = detector.getId()
            residual = butler.get('verifyBiasProc', instrument=cameraName, 
                                  exposure=exposureId, detector=detId)
            detStats = butler.get('verifyBiasDetStats', instrument=cameraName, 
                                  exposure=exposureId, detector=detId)
            display.mtv(residual)    
            display.scale('linear', 'zscale', None)

            continueDisplay, skipNumber = utils.interactiveBlock(f"{exposureId} {detector.getName()}", 
                                                                 detStats)
            if continueDisplay is False:
                break
        if continueDisplay is False:
            break